Esta es una demostración de cómo se va a realizar el proceso de ETL en el módulo ETL.py.
Por lo tanto, se va a trabajar únicamente con un dataframe a modo de muestra.


In [27]:
import pandas as pd

In [28]:
df_amazon = pd.read_csv('/datasets/amazon_prime_titles.csv')

In [29]:
df_amazon.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

Se definen qué columnas se van a utilizar para las consultas pertinentes.

*show_id*: Número de ID para cada serie o película <br>
*type*: Define si es serie o película <br>
*title*: Título <br>
*cast*: Reparto de actores y actrices <br>
*release_year*: Fecha de lanzamiento <br>
*duration*: Duración de la misma (en minutos o en temporadas) <br>
*listed_in*: Categoría/s <br>
 
Ya que no son necesarias para ninguna consulta, se descartan las siguientes columnas:

*director*: Nombre del director<br>
*country*: País de realización<br>
*date_added*: Fecha en que fueron introducidos los datos al dataset<br>
*rating*: Clasificación de la película (ATP, +13, etc)<br>
*description*: Una breve sinopsis<br>

In [30]:
#Se guardan las columnas a utilizar en una lista

columns = ['show_id', 'type', 'title', 'cast',
       'release_year', 'duration', 'listed_in']

In [31]:
#Se guarda una copia de cada dataset

df_amazon_copy = df_amazon.copy()

In [32]:
#Si las columnas no están en la lista columns, se dropean del dataframe.

df_amazon_copy.drop(columns=[col for col in df_amazon_copy if col not in columns], inplace=True)

In [33]:
#Podemos ver que no es tan grande la cantida de nulos, superando apenas el 10% solo en la columna cast
# por lo que no es necesario dropear nulos ni llenarlos (de momento) cruzando datos con las demás plataformas

df_amazon_copy.isna().sum()

show_id            0
type               0
title              0
cast            1233
release_year       0
duration           0
listed_in          0
dtype: int64

In [34]:
#Comprobamos con el output de la tabla anterior si hay duplicados, vemos que no los hay

df_amazon_copy.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9668 entries, 0 to 9667
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   cast          8435 non-null   object
 4   release_year  9668 non-null   int64 
 5   duration      9668 non-null   object
 6   listed_in     9668 non-null   object
dtypes: int64(1), object(6)
memory usage: 604.2+ KB


In [35]:
df_amazon_copy['listed_in'].str.contains('Comedy').sum()

2099

In [36]:
#Vamos a separar duration en cantidad, y luego en la unidad en la que está medido(minutos, temporadas)

df_amazon_copy

,show_id,type,title,cast,release_year,duration,listed_in
0,s1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113 min,"Comedy, Drama"
1,s2,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110 min,"Drama, International"
2,s3,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74 min,"Action, Drama, Suspense"
3,s4,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69 min,Documentary
4,s5,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45 min,"Drama, Fantasy"
...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,"Leo Gorcey, Bobby Jordan",1940,60 min,Comedy
9664,s9665,TV Show,Planet Patrol,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",2018,4 Seasons,TV Shows
9665,s9666,Movie,Outpost,"Ray Stevenson, Julian Wadham, Richard Brake, M...",2008,90 min,Action
9666,s9667,TV Show,Maradona: Blessed Dream,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",2021,1 Season,"Drama, Sports"


In [37]:
#Observamos que se separa la unidad por un espacio en blanco
df_amazon_copy.duration

0         113 min
1         110 min
2          74 min
3          69 min
4          45 min
          ...    
9663       60 min
9664    4 Seasons
9665       90 min
9666     1 Season
9667      103 min
Name: duration, Length: 9668, dtype: object

In [38]:
# El método str.split divide la columna en dos, separando el contenido, por defecto con un espacio, y con
# el parámetro expand=True, se expande en un dataframe con dos columnas.
# Luego el método join hace la unión del df original con las columnas.

df_amazon_copy = df_amazon_copy.join(df_amazon_copy.duration.str.split(expand=True))

In [39]:
# Dropeamos la vieja columna duration
# Renombramos las dos columnas nuevas, una con duration(cantidad de minutos o temporadas)
# y la otra con unit(unidad de tiempo, minutos o temporadas)

df_amazon_copy.drop(columns='duration', inplace=True)
df_amazon_copy.rename(columns={0:'duration',
                                1:'unit'}, inplace=True)
df_amazon_copy.sample(5)

,show_id,type,title,cast,release_year,listed_in,duration,unit
440,s441,Movie,The Special,"Davy Raphaely, Sarah French, Davi Sheridan, Su...",2020,"Drama, Horror, Suspense",94,min
1000,s1001,Movie,Simple Gifts: The Chamber Music Society at Sha...,NaN,2016,Documentary,84,min
57,s58,TV Show,Xploration Earth 2050,Chuck Pell,2017,"Documentary, Science Fiction",2,Seasons
2233,s2234,TV Show,Heavenly Sword and Dragon Slaying Sabre,"Zeng Shunxi, Chen Yuqi",2019,"Action, Adventure, Drama",1,Season
1263,s1264,TV Show,PRIME JAPAN (4K UHD),Jonathan Sherr,2017,TV Shows,1,Season


In [40]:
#Agregamos a qué plataforma pertenece este dataframe

df_amazon_copy['platform'] = 'Amazon'

In [41]:
# Y por último aplicamos el método title para normalizar mayúsculas y minúsculas
# Salteamos duration y release_year porque son números y no se puede aplicar este método
for n in df_amazon_copy:
    if n=='duration' or n=='release_year':
        pass
    else:
       df_amazon_copy[f'{n}'] = df_amazon_copy[f'{n}'].str.title()

In [42]:
# Asignamos el tipo de dato de duration 

df_amazon_copy.duration = df_amazon_copy.duration.astype('int64')

In [43]:
df_amazon_copy.head()

,show_id,type,title,cast,release_year,listed_in,duration,unit,platform
0,S1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,"Comedy, Drama",113,Min,Amazon
1,S2,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,"Drama, International",110,Min,Amazon
2,S3,Movie,Secrets Of Deception,"Tom Sizemore, Lorenzo Lamas, Robert Lasardo, R...",2017,"Action, Drama, Suspense",74,Min,Amazon
3,S4,Movie,Pink: Staying True,"Interviews With: Pink, Adele, Beyoncé, Britney...",2014,Documentary,69,Min,Amazon
4,S5,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,"Drama, Fantasy",45,Min,Amazon


Terminamos! Tenemos el df normalizado en todos sus datos. Y únicamente tenemos las columnas necesarias.

Luego, cuando juntemos los 4 DF, vamos a cambiar show_id para que no se repitan y lo podamos utilizar como id único.
